## install & imports

In [1]:
!apt-get update -qq
!apt-get install -qq -y wget unzip

!pip install --quiet gradio transformers torch pillow pyngrok


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 28.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.6/324.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.0 MB/s eta 0:00:00:00:0100:01


In [2]:
import os
import torch
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torchvision.models import ResNet50_Weights, VGG19_Weights, VGG16_Weights, Inception_V3_Weights, inception_v3
from torch.cuda.amp import GradScaler, autocast
from datetime import datetime
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
import pandas as pd
import matplotlib.pyplot as plt
from pyngrok import ngrok
import gradio as gr
import re
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import threading
import torch.nn.functional as F
import numpy as np
from transformers import BertTokenizerFast # BERT tokenizer

2025-08-25 17:58:44.975521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756144725.174311      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756144725.226624      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
NGROK_AUTH_TOKEN = #PASTE YOUR OWN TOKEN

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("ngrok authtoken configured.")

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## vocab

In [4]:
TRAIN_ANNOTATIONS_FOR_VOCAB = "/kaggle/input/all-data-coco/turkish_captions_train (1).csv"
FREQ_THRESHOLD_FOR_VOCAB = 2

In [5]:
def preprocess_caption_turkish(caption):
    caption = caption.strip()
    caption = caption.replace("I", "ı").replace("İ", "i").lower()
    caption = re.sub(r"[^a-z0-9ğüşıöç\s]", "", caption, flags=re.UNICODE)
    caption = re.sub(r'\s+', ' ', caption)
    return caption

In [6]:
class Vocabulary:
    def __init__(self, freq_threshold=1):
        self.freq_threshold = freq_threshold
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer(text):
        if not isinstance(text, str):
            return []
        return text.lower().split()

    def build_vocabulary(self, sentence_list_raw):
        frequencies = {}
        idx = len(self.itos)

        print(f"Kelime dağarcığı için {len(sentence_list_raw)} ham cümle işleniyor...")
        for sentence in tqdm(sentence_list_raw, desc="Kelime dağarcığı oluşturuluyor"):
            cleaned_sentence = preprocess_caption_turkish(sentence)
            for token in cleaned_sentence.split(): 
                if token: # Boş string tokenları (eğer oluşursa) atla
                    frequencies[token] = frequencies.get(token, 0) + 1
        
        print("Frekanslar hesaplandı. Kelimeler eşiğe göre kelime dağarcığına ekleniyor...")
        for token, count in frequencies.items():
            if count >= self.freq_threshold: 
                if token not in self.stoi:
                    self.stoi[token] = idx
                    self.itos[idx] = token
                    idx += 1
        print(f"Kelime dağarcığı oluşturma tamamlandı. Toplam boyut (özel tokenlar dahil): {len(self.itos)}")
        print(f"Eklenen kelime sayısı (özel tokenlar hariç): {idx - 4}")

    def numericalize(self, text):
        tokenized_text = self.tokenizer(text)
        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]



class CocoDataset(Dataset):
    def __init__(self, root_dir, captions_file, transform=None, vocab=None, 
                 freq_threshold_for_build=2, secondary_dir=None, max_samples=None): # vocab eklendi
        self.root_dir = root_dir
        self.secondary_dir = secondary_dir
        self.transform = transform
        self.captions_file_path = captions_file
        
        self.df = pd.read_csv(
            captions_file, on_bad_lines='skip', engine='python', encoding='utf-8'
        )
        self.df.dropna(subset=["Türkçe_açıklama"], inplace=True)
        self.df["filename"] = self.df["Coco_url"].apply(
            lambda x: x.split("/")[-1] if isinstance(x, str) else ""
        )
        
        self.df = self.df[self.df["filename"] != ""].copy()

        if max_samples is not None and max_samples > 0:
            if max_samples < len(self.df):
                self.df = self.df.head(max_samples).reset_index(drop=True)

        self.imgs = self.df["filename"]
        self.captions = self.df["Türkçe_açıklama"]

        if vocab is not None:
            self.vocab = vocab
        else:
            print(f"'{self.captions_file_path}' üzerinden YENİ kelime dağarcığı oluşturuluyor (freq_threshold={freq_threshold_for_build})...")
            self.vocab = Vocabulary(freq_threshold_for_build)
            self.vocab.build_vocabulary(self.captions.tolist())
            print(f"Yeni kelime dağarcığı oluşturuldu. Boyut: {len(self.vocab)}")
            
    # __len__ ve __getitem__ metodları aynı kalabilir
    # ... (geri kalan CocoDataset metodları)
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        caption = self.captions.iloc[index]
        img_id  = self.imgs.iloc[index]

        img_path = os.path.join(self.root_dir, img_id)
        if not os.path.exists(img_path) and self.secondary_dir is not None:
            img_path = os.path.join(self.secondary_dir, img_id)
        
        try:
            img = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            print(f"UYARI: Görüntü bulunamadı {img_path}")
            if index + 1 < len(self.df):
                print(f"{img_path} bulunamadı, bir sonraki örneğe geçiliyor.")
                return self.__getitem__(index + 1)
            else:
                raise FileNotFoundError(f"Kritik: Görüntü {img_path} bulunamadı ve başka örnek yok.")


        if self.transform is not None:
            img = self.transform(img)

        new_cap = preprocess_caption_turkish(caption)
        numericalized_caption = (
            [self.vocab.stoi["<SOS>"]]
            + self.vocab.numericalize(new_cap) # self.vocab artık paylaşılan vocab olacak
            + [self.vocab.stoi["<EOS>"]]
        )
        return img, torch.tensor(numericalized_caption, dtype=torch.long)

class MyCollate:
    def __init__(self, pad_token_id):
        self.pad_token_id = pad_token_id

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_token_id)
        return imgs, targets


def get_loader(
    root_folder,
    secondary_folder,
    annotation_file,
    transform,
    batch_size=64,
    num_workers=8,
    shuffle=True,
    pin_memory=True,
    max_samples=None,
    vocab=None,
    freq_threshold_for_build=5 
):
    dataset = CocoDataset(
        root_dir=root_folder,
        captions_file=annotation_file,
        transform=transform,
        secondary_dir=secondary_folder,
        max_samples=max_samples,
        vocab=vocab,
        freq_threshold_for_build=freq_threshold_for_build
    )
    
    if len(dataset) == 0:
        print(f"UYARI: {annotation_file} için oluşturulan veri seti BOŞ! max_samples={max_samples}")
    
    pad_token_id = dataset.vocab.stoi["<PAD>"]

    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory,
        collate_fn=MyCollate(pad_token_id=pad_token_id),
    )
    return loader, dataset


In [7]:
def load_vocab_from_train_file(annotations_file, freq_threshold, isRep = None):
    print(f"'{annotations_file}' üzerinden kelime dağarcığı yeniden oluşturuluyor...")
    df = pd.read_csv(annotations_file, on_bad_lines='skip', engine='python', encoding='utf-8')
    df.dropna(subset=["Türkçe_açıklama"], inplace=True)
    
    df["filename"] = df["Coco_url"].apply(
        lambda x: x.split("/")[-1] if isinstance(x, str) else ""
    )
    df = df[df["filename"] != ""].copy()
    if isRep == "reppen":
        df = df.drop_duplicates(subset=["filename"], keep="first").reset_index(drop=True)
        
    captions_list = df["Türkçe_açıklama"].tolist()
    
    vocab = Vocabulary(freq_threshold)
    vocab.build_vocabulary(captions_list)
    print(f"Kelime dağarcığı oluşturuldu. Boyut: {len(vocab)}")
    return vocab

In [8]:

vocabulary = load_vocab_from_train_file(TRAIN_ANNOTATIONS_FOR_VOCAB, FREQ_THRESHOLD_FOR_VOCAB)
rep_vocabulary = load_vocab_from_train_file(TRAIN_ANNOTATIONS_FOR_VOCAB, FREQ_THRESHOLD_FOR_VOCAB, "reppen")

'/kaggle/input/all-data-coco/turkish_captions_train (1).csv' üzerinden kelime dağarcığı yeniden oluşturuluyor...
Kelime dağarcığı için 336496 ham cümle işleniyor...


Kelime dağarcığı oluşturuluyor:   0%|          | 0/336496 [00:00<?, ?it/s]

Frekanslar hesaplandı. Kelimeler eşiğe göre kelime dağarcığına ekleniyor...
Kelime dağarcığı oluşturma tamamlandı. Toplam boyut (özel tokenlar dahil): 23233
Eklenen kelime sayısı (özel tokenlar hariç): 23229
Kelime dağarcığı oluşturuldu. Boyut: 23233
'/kaggle/input/all-data-coco/turkish_captions_train (1).csv' üzerinden kelime dağarcığı yeniden oluşturuluyor...
Kelime dağarcığı için 75214 ham cümle işleniyor...


Kelime dağarcığı oluşturuluyor:   0%|          | 0/75214 [00:00<?, ?it/s]

Frekanslar hesaplandı. Kelimeler eşiğe göre kelime dağarcığına ekleniyor...
Kelime dağarcığı oluşturma tamamlandı. Toplam boyut (özel tokenlar dahil): 11528
Eklenen kelime sayısı (özel tokenlar hariç): 11524
Kelime dağarcığı oluşturuldu. Boyut: 11528


## tokenizer

In [9]:
BERT_MODEL_NAME = "dbmdz/bert-base-turkish-cased"

def preprocess_caption_for_bert_turkish(caption: str) -> str:
    caption = caption.strip()
    caption = caption.replace("I", "ı").replace("İ", "i")
    return caption

In [10]:
# tokenizer'li her seyin sonuna T ekledik. 
class VocabularyT:
    def __init__(self, tokenizer_name: str = BERT_MODEL_NAME):
        print(f"'{tokenizer_name}' BERT tokenizer yükleniyor...")
        try:
            self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_name)
        except Exception as e:
            print(f"HATA: BERT tokenizer '{tokenizer_name}' yüklenemedi. Hata: {e}")
            print("Lütfen 'transformers' kütüphanesinin kurulu ve model adının doğru olduğundan emin olun.")
            raise

        self.pad_token = self.tokenizer.pad_token
        self.sos_token = self.tokenizer.cls_token
        self.eos_token = self.tokenizer.sep_token 
        self.unk_token = self.tokenizer.unk_token

        self.pad_token_id = self.tokenizer.pad_token_id
        self.sos_token_id = self.tokenizer.cls_token_id
        self.eos_token_id = self.tokenizer.sep_token_id
        self.unk_token_id = self.tokenizer.unk_token_id

        print(f"BERT Tokenizer '{tokenizer_name}' başarıyla yüklendi.")
        print(f"  PAD token: '{self.pad_token}' (ID: {self.pad_token_id})")
        print(f"  SOS/CLS token: '{self.sos_token}' (ID: {self.sos_token_id})")
        print(f"  EOS/SEP token: '{self.eos_token}' (ID: {self.eos_token_id})")
        print(f"  UNK token: '{self.unk_token}' (ID: {self.unk_token_id})")
        print(f"  Vocabulary size: {self.tokenizer.vocab_size}")

        self._itos_list = None 

    def __len__(self):
        return self.tokenizer.vocab_size

    def numericalize(self, text_processed_for_bert: str, add_special_tokens: bool = False) -> list[int]:
        token_ids = self.tokenizer.encode(
            text_processed_for_bert,
            add_special_tokens=add_special_tokens,
            truncation=False, # Truncation ve max_length genellikle encode_plus'ta yönetilir
        )
        return token_ids

    @property
    def itos(self):
        if self._itos_list is None:
            self._itos_list = self.tokenizer.convert_ids_to_tokens(list(range(self.tokenizer.vocab_size)))
        return self._itos_list

    @property
    def stoi(self):
        return self.tokenizer.get_vocab()
    def decode_ids_to_string(self, token_ids: list[int]) -> str:
        return self.tokenizer.decode(token_ids, skip_special_tokens=True)

class CocoDatasetT(Dataset):
    def __init__(self, root_dir: str, captions_file: str, transform=None, vocab_tokenizer: VocabularyT = None,
                 secondary_dir: str = None, max_samples: int = None, max_caption_len: int = 100):
        self.root_dir = root_dir
        self.secondary_dir = secondary_dir
        self.transform = transform
        self.captions_file_path = captions_file
        self.max_caption_len = max_caption_len

        df_initial = pd.read_csv(
            captions_file, on_bad_lines='skip', engine='python', encoding='utf-8'
        )
        df_initial.dropna(subset=["Türkçe_açıklama"], inplace=True)

        if "filename" not in df_initial.columns and "Coco_url" in df_initial.columns:
            df_initial["filename"] = df_initial["Coco_url"].apply(
                lambda x: x.split("/")[-1] if isinstance(x, str) else ""
            )
        elif "filename" not in df_initial.columns and "image" in df_initial.columns: # Flickr8k benzeri format için
            df_initial = df_initial.rename(columns={"image": "filename"})

        df_initial = df_initial[df_initial["filename"] != ""].copy()

        self.df = df_initial.reset_index(drop=True)

        if max_samples is not None and max_samples > 0:
            # max_samples artık toplam başlık sayısı üzerinden uygulanacak.
            if max_samples < len(self.df):
                self.df = self.df.head(max_samples).reset_index(drop=True)
        
        print(f"Veri kümesi hazırlandı. Toplam görsel-başlık çifti sayısı: {len(self.df)}")

        self.imgs_filenames = self.df["filename"].tolist()
        self.raw_captions_list = self.df["Türkçe_açıklama"].tolist()

        if vocab_tokenizer is not None:
            self.vocab_tokenizer = vocab_tokenizer
        else:
            print(f"'{self.captions_file_path}' için YENİ BERT tokenizer ve vocab oluşturuluyor (CocoDataset içinde)...")
            # BERT_MODEL_NAME değişkeninin tanımlı olması gerekir
            self.vocab_tokenizer = VocabularyT(tokenizer_name=BERT_MODEL_NAME)
            
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index: int):
        caption_text_raw = self.raw_captions_list[index]
        img_filename = self.imgs_filenames[index]

        img_path = os.path.join(self.root_dir, img_filename)
        if self.secondary_dir and not os.path.exists(img_path):
            img_path = os.path.join(self.secondary_dir, img_filename)

        try:
            img = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            print(f"UYARI: __getitem__ içinde görüntü bulunamadı: {img_path}.")
            raise FileNotFoundError(f"Görüntü yüklenemedi: {img_path}")

        if self.transform is not None:
            img = self.transform(img)
            
        processed_caption_for_bert = preprocess_caption_for_bert_turkish(caption_text_raw)
        
        encoding = self.vocab_tokenizer.tokenizer.encode_plus(
            processed_caption_for_bert,
            add_special_tokens=True,
            max_length=self.max_caption_len,
            padding='do_not_pad', # Padding collate_fn'de yapılacak
            truncation=True,
            return_attention_mask=False,
            return_token_type_ids=False,
            return_tensors=None 
        )
        numericalized_caption_ids = encoding['input_ids']

        return img, torch.tensor(numericalized_caption_ids, dtype=torch.long)

class MyCollateT:
    def __init__(self, pad_token_id: int):
        self.pad_token_id = pad_token_id

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_token_id)
        return imgs, targets

def get_loader(
    root_folder: str,
    secondary_folder: str,
    annotation_file: str,
    transform,
    batch_size: int = 64,
    num_workers: int = 8,
    shuffle: bool = True,
    pin_memory: bool = True,
    max_samples: int = None,
    vocab_tokenizer: VocabularyT = None,
    max_caption_len: int = 100      
):
    dataset = CocoDatasetT(
        root_dir=root_folder,
        captions_file=annotation_file,
        transform=transform,
        secondary_dir=secondary_folder,
        max_samples=max_samples,
        vocab_tokenizer=vocab_tokenizer,
        max_caption_len=max_caption_len
    )

    if len(dataset) == 0:
        print(f"UYARI: {annotation_file} için oluşturulan veri seti BOŞ! max_samples={max_samples}")
        return None, None

    pad_token_id = dataset.vocab_tokenizer.pad_token_id

    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory,
        collate_fn=MyCollateT(pad_token_id=pad_token_id),
    )
    return loader, dataset

In [11]:
vocabT = VocabularyT(BERT_MODEL_NAME)

'dbmdz/bert-base-turkish-cased' BERT tokenizer yükleniyor...


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

BERT Tokenizer 'dbmdz/bert-base-turkish-cased' başarıyla yüklendi.
  PAD token: '[PAD]' (ID: 0)
  SOS/CLS token: '[CLS]' (ID: 2)
  EOS/SEP token: '[SEP]' (ID: 3)
  UNK token: '[UNK]' (ID: 1)
  Vocabulary size: 32000


## models


In [12]:
# CHECKPOINTS FOR MODELS
BNL_MINI_CHECKPOINT = "/kaggle/input/inception_lstm_vocab/pytorch/default/1/inception_lstm_vocab.pth"
BNL_LARGE_CHECKPOINT = "/kaggle/input/resnet50_lstm_combined/pytorch/default/1/resnet50_combined.pth" 
BNL_MID_CHECKPOINT = "/kaggle/input/vgg16_lstm_combined/pytorch/default/1/vgg16_combined_model.pth"
BNL_LARGE_REP_CHECKPOINT = "/kaggle/input/rep_penalty/pytorch/default/1/mscoco_checkpoint_best(7).pth"
BNL_LARGE_TURBO_CHECKPOINT = "/kaggle/input/resnet_lstm_tokenizer/pytorch/default/1/resnet_lstm_tokenizer.pth"

BNT_LARGE_CHECKPOINT = "/kaggle/input/resnet50_transformer_vocab/pytorch/default/1/mscoco_transformer_checkpoint_best(2).pth"
BNT_LARGE_TURBO_CHECKPOINT = "/kaggle/input/resnet_transformer_tokenizer/pytorch/default/1/resnet_transformer_tokenizer.pth"
BNT_LARGE_ULTRA_CHECKPOINT = "/kaggle/input/inception_transformer_tokenizer/pytorch/default/1/inception_transformer_tokenizer.pth"

In [13]:

EMBED_SIZE_LSTM = 512
HIDDEN_SIZE_LSTM = 512
NUM_LAYERS_LSTM = 2

In [14]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size: int, cnn_type: str, train_CNN: bool = False):
        super(EncoderCNN, self).__init__()
        self.cnn_type = cnn_type.lower()
        self.train_CNN = train_CNN

        if "tokenizer" in self.cnn_type:
            weights = ResNet50_Weights.DEFAULT
            self.resnet = resnet50(weights=weights)
            if not self.train_CNN:
                for param in self.resnet.parameters():
                    param.requires_grad = False
            in_features_resnet = self.resnet.fc.in_features 
            self.resnet.fc = nn.Linear(in_features_resnet, embed_size)

        elif "resnet50" in self.cnn_type:
            resnet = models.resnet50(pretrained=True)
            self.resnet = nn.Sequential(*list(resnet.children())[:-1])
            self.output_size = 2048
            self.relu = nn.ReLU()
            for p in self.resnet.parameters(): p.requires_grad = train_CNN

        elif "vgg16" in self.cnn_type:
            vgg = models.vgg16(pretrained=True)
            self.vgg = vgg.features
            self.output_size = 512
            self.relu = nn.ReLU()
            for p in self.vgg.parameters(): p.requires_grad = train_CNN

        elif "inception" in self.cnn_type:
            weights = Inception_V3_Weights.DEFAULT
            self.inception = inception_v3(weights=weights, aux_logits=True)
            self.inception.aux_logits = False
            self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
            self.output_size = embed_size
            self.relu = nn.ReLU()

        if "resnet" in self.cnn_type or "vgg" in self.cnn_type:
            self.linear = nn.Linear(self.output_size, embed_size)
        # ortak aktivasyon + dropout
        
        self.dropout = nn.Dropout(0.5)

    def forward(self, images: torch.Tensor) -> torch.Tensor:
        if "resnet" in self.cnn_type:
            x = self.resnet(images)      
            x = x.view(x.size(0), -1)      
            x = self.linear(x)             

        elif "inception" in self.cnn_type:
            x = self.inception(images)    
        elif "tokenizer" in self.cnn_type:
            features = self.resnet(images)
            return self.dropout(features)
        else:  # vgg
            x = self.vgg(images)          
            x = x.mean(dim=[2,3])         
            x = self.linear(x)            

        x = self.relu(x)
        x = self.dropout(x)
        return x


In [15]:
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, dropout_p=0.5):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.embed_dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(
            embed_size,
            hidden_size,
            num_layers,
            dropout=dropout_p if num_layers > 1 else 0.0
        )
        self.output_dropout = nn.Dropout(dropout_p)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, features, captions, lengths=None):
        embeddings = self.embed(captions)
        embeddings = self.embed_dropout(embeddings)
        embeddings = embeddings.permute(1, 0, 2)
        embeddings = torch.cat((features.unsqueeze(0), embeddings), dim=0)

        if lengths is not None:
            packed_lengths = [l + 1 for l in lengths]
            packed = pack_padded_sequence(embeddings, packed_lengths, enforce_sorted=False)
            packed_outputs, _ = self.lstm(packed)
            hiddens, _ = pad_packed_sequence(packed_outputs)
        else:
            hiddens, _ = self.lstm(embeddings)                                   

        hiddens = self.output_dropout(hiddens)
        outputs = self.linear(hiddens)                                             
        outputs = outputs[1:]                                                     
        outputs = outputs.permute(1, 0, 2)                                         

        return outputs

In [16]:
class CNNtoRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, vocab, num_layers, cnn_type): # Changed default to resnet50
        super(CNNtoRNN, self).__init__()
        self.encoderCNN = EncoderCNN(embed_size, cnn_type) # Pass cnn_type
        self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

    def forward(self, images, captions, lengths=None):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features, captions, lengths)
        return outputs

    def caption_image(self, image, vocab, max_length=50):
        result_caption = []

        with torch.no_grad():
            x = self.encoderCNN(image).unsqueeze(0)
            states = None

            for _ in range(max_length):
                hiddens, states = self.decoderRNN.lstm(x, states)
                output = self.decoderRNN.linear(hiddens.squeeze(0))
                predicted = output.argmax(1)
                result_caption.append(predicted.item())
                x = self.decoderRNN.embed(predicted).unsqueeze(0)

                if vocabulary.itos[predicted.item()] == "<EOS>":
                    break

        return [vocabulary.itos[idx] for idx in result_caption]

In [17]:
class DecoderRNN_TOK(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, dropout_p=0.5):
        super(DecoderRNN_TOK, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.embed_dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(
            embed_size,
            hidden_size,
            num_layers,
            dropout=dropout_p if num_layers > 1 else 0.0
        )
        self.output_dropout = nn.Dropout(dropout_p)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, features, captions, lengths=None): # lengths artık kullanılmayacak
        embeddings = self.embed(captions)
        embeddings = self.embed_dropout(embeddings)

        embeddings = embeddings.permute(1, 0, 2)
        embeddings = torch.cat(
            (features.unsqueeze(0), embeddings), dim=0
        )

        hiddens, _ = self.lstm(embeddings)

        hiddens = self.output_dropout(hiddens)
        outputs = self.linear(hiddens)
        outputs = outputs[1:]
        outputs = outputs.permute(1, 0, 2)

        return outputs

In [18]:
class CNNtoRNN_TOK(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, vocab, num_layers, cnn_type="tokenizer"): # vocab_size BERT'ten gelecek
        super(CNNtoRNN_TOK, self).__init__()
        self.encoderCNN = EncoderCNN(embed_size, cnn_type="tokenizer") # EncoderCNN (ResNet50'li versiyon)
        self.decoderRNN = DecoderRNN_TOK(embed_size, hidden_size, vocab_size, num_layers)

    def forward(self, images, captions, lengths=None):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features, captions, lengths)
        return outputs

    def caption_image(self, image_tensor: torch.Tensor, vocab: VocabularyT, max_length=50) -> list[str]:
        result_ids = []
        self.eval()

        with torch.no_grad():
            features = self.encoderCNN(image_tensor)
            states = None
            x = features.unsqueeze(0)
            _, states = self.decoderRNN.lstm(x, states)
            sos_id_tensor = torch.tensor([vocab.sos_token_id], device=features.device)
            x = self.decoderRNN.embed(sos_id_tensor).unsqueeze(0) # Shape: (1, 1, embed_size)

            for _ in range(max_length):
                hiddens, states = self.decoderRNN.lstm(x, states) # hiddens: (1, 1, hidden_size)
                
                output_logits = self.decoderRNN.linear(hiddens.squeeze(0)) # (1, vocab_size)
                
                predicted_id = output_logits.argmax(1).item()
                if predicted_id == vocab.eos_token_id:
                    break
                
                result_ids.append(predicted_id)
                
                next_word_tensor = torch.tensor([predicted_id], device=features.device)
                x = self.decoderRNN.embed(next_word_tensor).unsqueeze(0)

        caption_tokens = [vocab.itos[idx] for idx in result_ids]
        return caption_tokens

In [19]:
class CNNtoRNN_REP(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, model_type="resnet50"): # Changed default to resnet50
        super(CNNtoRNN_REP, self).__init__()
        self.encoderCNN = EncoderCNN(embed_size, model_type) # Pass cnn_type
        self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

    def forward(self, images, captions, lengths=None):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features, captions, lengths)
        return outputs

    def caption_image(self, image, vocabulary=rep_vocabulary, max_length=50):
        result_caption = []

        with torch.no_grad():
            x = self.encoderCNN(image).unsqueeze(0)
            states = None

            for _ in range(max_length):
                hiddens, states = self.decoderRNN.lstm(x, states)
                output = self.decoderRNN.linear(hiddens.squeeze(0))
                predicted = output.argmax(1)
                result_caption.append(predicted.item())
                x = self.decoderRNN.embed(predicted).unsqueeze(0)

                if vocabulary.itos[predicted.item()] == "<EOS>":
                    break

        return [vocabulary.itos[idx] for idx in result_caption]

    def decoder_step(self, input_token, hidden_state):
        emb = self.decoderRNN.embed(input_token)  
        emb = emb.unsqueeze(1)                    

        output, new_hidden = self.decoderRNN.lstm(emb, hidden_state)
        logits = self.decoderRNN.linear(output.squeeze(0))  
        return logits, new_hidden

    def caption_image_beam_search(
        self,
        image_tensor,         
        vocabulary=rep_vocabulary,
        beam_size=5,
        max_length=20,
        repetition_penalty=1.2,
        ngram_block_size=3,
        length_penalty=1.0
    ):
        device = next(self.parameters()).device
        self.eval()

        with torch.no_grad():
            features = self.encoderCNN(image_tensor.to(device))  

        vocab_stoi = vocabulary.stoi
        vocab_itos = vocabulary.itos
        start_token_id = vocab_stoi["<SOS>"]
        eos_token_id   = vocab_stoi["<EOS>"]
        vocab_size     = len(vocabulary)

        beams = []
        beams.append({
            'sequence': [start_token_id],
            'score': 0.0,
            'normalized_score': 0.0,
            'hidden': None, 
            'cell': None
        })
        completed_beams = []

        for step in range(max_length):
            all_candidates = []

            for beam in beams:
                last_token = beam['sequence'][-1]

                if last_token == eos_token_id:
                    completed_beams.append(beam)
                    continue

                curr_token_tensor = torch.tensor([last_token], device=device)

                hidden_state = None
                if beam['hidden'] is not None:
                    hidden_state = (beam['hidden'], beam['cell'])

                with torch.no_grad():
                    if step == 0:
                        x0 = features.unsqueeze(0)  # (1, 1, embed_size)
                        output, new_hidden = self.decoderRNN.lstm(x0, None)
                        logits = self.decoderRNN.linear(output.squeeze(0))  
                    else:
                        logits, new_hidden = self.decoder_step(curr_token_tensor, hidden_state)

                logits = logits.squeeze(0)  

                if len(beam['sequence']) > 0:
                    token_counts = {}
                    for tkn in beam['sequence']:
                        token_counts[tkn] = token_counts.get(tkn, 0) + 1
                    for tkn, cnt in token_counts.items():
                        if tkn < vocab_size and cnt > 0:
                            logits[tkn] = logits[tkn] / (repetition_penalty ** cnt)


                if len(beam['sequence']) >= ngram_block_size - 1:
                    prev_ngrams = set()
                    seq = beam['sequence']
                    n = ngram_block_size - 1
                    for i in range(len(seq) - n + 1):
                        prev_ngrams.add(tuple(seq[i:i+n]))
                    blocked = set()
                    for ngram in prev_ngrams:
                        for candidate_id in range(vocab_size):
                            new_ngram = ngram + (candidate_id,)
                            if new_ngram in set(tuple(seq[i:i+ngram_block_size]) 
                                               for i in range(len(seq) - ngram_block_size + 1)):
                                blocked.add(candidate_id)
                    for bt in blocked:
                        logits[bt] = -float('inf')

                probs = F.softmax(logits, dim=-1)   
                top_probs, top_indices = torch.topk(probs, beam_size)

                for i in range(beam_size):
                    token_id = top_indices[i].item()
                    prob = top_probs[i].item()

                    new_seq = beam['sequence'] + [token_id]
                    new_score = beam['score'] + np.log(prob + 1e-8)
                    length_norm = (len(new_seq) ** length_penalty)
                    new_norm_score = new_score / length_norm

                    cand = {
                        'sequence': new_seq,
                        'score': new_score,
                        'normalized_score': new_norm_score,
                        'hidden': new_hidden[0],  
                        'cell': new_hidden[1]     
                    }
                    all_candidates.append(cand)

            all_candidates.sort(key=lambda x: x['normalized_score'], reverse=True)
            beams = []
            for cand in all_candidates:
                if len(beams) >= beam_size:
                    break
                if cand['sequence'][-1] == eos_token_id:
                    completed_beams.append(cand)
                else:
                    beams.append(cand)

            if len(beams) == 0:
                break

        if len(completed_beams) == 0:
            completed_beams = beams

        best_beam = max(completed_beams, key=lambda x: x['normalized_score'])
        seq_ids = best_beam['sequence']  

        if seq_ids[0] == start_token_id:
            seq_ids = seq_ids[1:]
        if eos_token_id in seq_ids:
            seq_ids = seq_ids[:seq_ids.index(eos_token_id)]
        caption_words = [vocab_itos[idx] if idx in vocab_itos else "<UNK>" for idx in seq_ids]
        return caption_words


## transformers

In [21]:
import torch
import torch.nn as nn
import statistics
import torchvision.models as models
from torchvision.models import resnet50, ResNet50_Weights, inception_v3
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import math
from torchvision.models.feature_extraction import create_feature_extractor

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1) 
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class ImageFeatureExtractor(nn.Module):
    def __init__(self, d_model, cnn_type, train_CNN=False):
        super(ImageFeatureExtractor, self).__init__()
        self.train_CNN = train_CNN
        if "inception" in cnn_type:
            weights = Inception_V3_Weights.DEFAULT
            inception_model_full = inception_v3(weights=weights) 
            inception_model_full.aux_logits = False 

            return_nodes = {'Mixed_7c': 'spatial_features'} 
            self.feature_extractor_backbone = create_feature_extractor(inception_model_full, return_nodes=return_nodes)
        else:
            weights = ResNet50_Weights.DEFAULT
            resnet_model = resnet50(weights=weights)

            return_nodes = {'layer4': 'spatial_features'}
            self.feature_extractor_backbone = create_feature_extractor(
                resnet_model,
                return_nodes=return_nodes
            )

        self.feature_proj = nn.Conv2d(2048, d_model, kernel_size=1)

        if not self.train_CNN:
            for param in self.feature_extractor_backbone.parameters():
                param.requires_grad = False

    def forward(self, images):
        extracted_outputs = self.feature_extractor_backbone(images)
        spatial_features = extracted_outputs['spatial_features']
        projected_features = self.feature_proj(spatial_features)

        batch_size, d_model_channels, h_out, w_out = projected_features.shape
        features_seq = projected_features.view(batch_size, d_model_channels, h_out * w_out)
        features_seq = features_seq.permute(0, 2, 1)

        return features_seq 

class ImageTransformerEncoder(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, dim_feedforward, dropout=0.1):
        super(ImageTransformerEncoder, self).__init__()
        self.d_model = d_model
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, 
                                                   dim_feedforward=dim_feedforward, dropout=dropout, 
                                                   batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)

    def forward(self, src_img_features, src_padding_mask=None):
        src_img_features_permuted = src_img_features.permute(1, 0, 2)
        src_with_pe = self.pos_encoder(src_img_features_permuted)
        src_with_pe_batch_first = src_with_pe.permute(1, 0, 2)
        memory = self.transformer_encoder(src_with_pe_batch_first, src_key_padding_mask=src_padding_mask)     
        return memory

class TextTransformerDecoder(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_decoder_layers, dim_feedforward, dropout=0.1, max_seq_len=100):
        super(TextTransformerDecoder, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len=max_seq_len) 
        
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead,
                                                   dim_feedforward=dim_feedforward, dropout=dropout,
                                                   batch_first=True) 
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, tgt_caption_tokens, memory, tgt_mask=None, 
                tgt_key_padding_mask=None, memory_key_padding_mask=None):
        
        tgt_emb = self.embedding(tgt_caption_tokens) * math.sqrt(self.d_model) 

        tgt_emb_permuted = tgt_emb.permute(1,0,2) 
        tgt_with_pe = self.pos_encoder(tgt_emb_permuted)
        tgt_with_pe_batch_first = tgt_with_pe.permute(1,0,2) 
        output = self.transformer_decoder(tgt=tgt_with_pe_batch_first, 
                                           memory=memory, 
                                           tgt_mask=tgt_mask,
                                           tgt_key_padding_mask=tgt_key_padding_mask,
                                           memory_key_padding_mask=memory_key_padding_mask)
        
        return self.fc_out(output)

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

class CaptioningTransformer(nn.Module):
    def __init__(self, num_tokens_vocab, d_model, nhead, num_encoder_layers, num_decoder_layers, 
                 dim_feedforward, dropout=0.1, max_seq_len=100, cnn_type="resnet50"):
        super(CaptioningTransformer, self).__init__()
        
        self.image_feature_extractor = ImageFeatureExtractor(d_model, cnn_type) 
        self.image_transformer_encoder = ImageTransformerEncoder(d_model, nhead, num_encoder_layers, dim_feedforward, dropout)
        self.text_transformer_decoder = TextTransformerDecoder(num_tokens_vocab, d_model, nhead, num_decoder_layers, 
                                                               dim_feedforward, dropout, max_seq_len)
        self.d_model = d_model
        self.num_tokens_vocab = num_tokens_vocab

    def _generate_padding_mask(self, sequences, pad_token_id):
        return (sequences == pad_token_id)

    def forward(self, images, captions_input, captions_target_padding_mask=None):
        img_spatial_features_seq = self.image_feature_extractor(images, cnn_type)
        image_memory = self.image_transformer_encoder(img_spatial_features_seq, src_padding_mask=None)
        tgt_seq_len = captions_input.size(1)
        device = captions_input.device
        tgt_mask = self.text_transformer_decoder.generate_square_subsequent_mask(tgt_seq_len).to(device)
        decoder_output_logits = self.text_transformer_decoder(
            tgt_caption_tokens=captions_input, 
            memory=image_memory, 
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=captions_target_padding_mask,
            memory_key_padding_mask=None
        )
        return decoder_output_logits

    @torch.no_grad()
    def generate_caption(self, image_tensor, vocabulary, start_token_id, end_token_id, pad_token_id, max_length=50):
        self.eval()
        device = image_tensor.device

        img_spatial_features_seq = self.image_feature_extractor(image_tensor)
        image_memory = self.image_transformer_encoder(img_spatial_features_seq)
        generated_ids = torch.tensor([[start_token_id]], dtype=torch.long, device=device)

        for _ in range(max_length - 1):
            tgt_seq_len = generated_ids.size(1)
            tgt_mask = self.text_transformer_decoder.generate_square_subsequent_mask(tgt_seq_len).to(device)
            
            tgt_padding_mask = self._generate_padding_mask(generated_ids, pad_token_id).to(device)
            output_logits = self.text_transformer_decoder(
                tgt_caption_tokens=generated_ids, 
                memory=image_memory, 
                tgt_mask=tgt_mask,
                tgt_key_padding_mask=tgt_padding_mask,
                memory_key_padding_mask=None
            )
            next_token_logits = output_logits[:, -1, :] 
            next_token_id = torch.argmax(next_token_logits, dim=-1) 
            generated_ids = torch.cat((generated_ids, next_token_id.unsqueeze(1)), dim=1)

            if next_token_id.item() == end_token_id:
                break
        
        output_tokens = [vocabulary.itos[idx.item()] for idx in generated_ids[0, 1:]]
        return output_tokens


In [22]:
class CaptioningTransformer_T(nn.Module):
    def __init__(self, num_tokens_vocab: int, d_model: int, nhead: int, num_encoder_layers: int, num_decoder_layers: int,
                 dim_feedforward: int, pad_token_id_for_masking: int, dropout: float = 0.1, max_seq_len: int = 100, cnn_type="resnet50"):
        super(CaptioningTransformer_T, self).__init__()
        self.image_feature_extractor = ImageFeatureExtractor(d_model, cnn_type=cnn_type)
        self.image_transformer_encoder = ImageTransformerEncoder(d_model, nhead, num_encoder_layers, dim_feedforward, dropout)
        self.text_transformer_decoder = TextTransformerDecoder(num_tokens_vocab, d_model, nhead, num_decoder_layers,
                                                               dim_feedforward, dropout, max_seq_len)
        self.d_model = d_model
        self.num_tokens_vocab = num_tokens_vocab
        self.pad_token_id_for_masking = pad_token_id_for_masking 

    def _generate_padding_mask(self, sequences: torch.Tensor) -> torch.Tensor:
        return (sequences == self.pad_token_id_for_masking) 

    def forward(self, images: torch.Tensor, captions_input: torch.Tensor, captions_target_padding_mask: torch.Tensor = None):
        img_spatial_features_seq = self.image_feature_extractor(images)
        image_memory = self.image_transformer_encoder(img_spatial_features_seq, src_padding_mask=None)

        tgt_seq_len = captions_input.size(1)
        device = captions_input.device
        tgt_mask = self.text_transformer_decoder.generate_square_subsequent_mask(tgt_seq_len).to(device)

        if captions_target_padding_mask is None:
            captions_target_padding_mask = self._generate_padding_mask(captions_input).to(device)

        decoder_output_logits = self.text_transformer_decoder(
            tgt_caption_tokens=captions_input,
            memory=image_memory,
            tgt_mask=tgt_mask, # Causal mask
            tgt_key_padding_mask=captions_target_padding_mask,
            memory_key_padding_mask=None 
        )
        return decoder_output_logits
    @torch.no_grad()
    def generate_caption(self, image_tensor: torch.Tensor, vocabulary_tokenizer: VocabularyT, max_length: int = 50) -> str:
        self.eval()
        device = image_tensor.device
        img_spatial_features_seq = self.image_feature_extractor(image_tensor)
        image_memory = self.image_transformer_encoder(img_spatial_features_seq)
        generated_ids = torch.tensor([[vocabulary_tokenizer.sos_token_id]], dtype=torch.long, device=device)

        for _ in range(max_length - 1):
            tgt_seq_len = generated_ids.size(1)
            tgt_mask = self.text_transformer_decoder.generate_square_subsequent_mask(tgt_seq_len).to(device)
            tgt_padding_mask = self._generate_padding_mask(generated_ids).to(device)
            output_logits = self.text_transformer_decoder(
                tgt_caption_tokens=generated_ids,
                memory=image_memory,
                tgt_mask=tgt_mask,
                tgt_key_padding_mask=tgt_padding_mask
            )
            next_token_logits = output_logits[:, -1, :]
            
            next_token_id = torch.argmax(next_token_logits, dim=-1)

            generated_ids = torch.cat([generated_ids, next_token_id.unsqueeze(1)], dim=1)
            if next_token_id.item() == vocabulary_tokenizer.eos_token_id:
                break
        
        caption = vocabulary_tokenizer.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        
        self.train() 
        
        return caption

In [23]:
def get_tokenizer_model_for_inference(checkpoint_path, vocab, cnn_type, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    if "config" in checkpoint:
        cfg = checkpoint["config"]
        print("[INFO] Checkpoint'ten config yüklendi.")
    else:
        print("[UYARI] Config bilgisi bulunamadı. Varsayılan yapılandırma kullanılıyor.")
        cfg = {
            "d_model": 256,
            "nhead": 8,
            "num_encoder_layers": 3,
            "num_decoder_layers": 3,
            "dim_feedforward": 1024,
            "transformer_dropout": 0.1,
            "vocab_size": len(vocab),
            "max_caption_len": 100
        }
    cfg["vocab_size"] = len(vocab)
    pad_token_id = vocab.pad_token_id
    model = CaptioningTransformer_T(
        num_tokens_vocab=cfg["vocab_size"],
        d_model=cfg["d_model"],
        nhead=cfg["nhead"],
        num_encoder_layers=cfg["num_encoder_layers"],
        num_decoder_layers=cfg["num_decoder_layers"],
        dim_feedforward=cfg["dim_feedforward"],
        dropout=cfg["transformer_dropout"],
        max_seq_len=cfg["max_caption_len"],
        pad_token_id_for_masking=pad_token_id,
        cnn_type=cnn_type
    ).to(device)

    state_dict = checkpoint.get("state_dict", checkpoint)
    model.load_state_dict(state_dict)
    print("[✓] Model ağırlıkları başarıyla yüklendi.")

    return model

In [24]:
def generate_caption_from_image_path_T(image_path, model, tokenizer, device):
    weights = ResNet50_Weights.DEFAULT
    preprocess = weights.transforms()
    image = Image.open(image_path).convert("RGB")
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        caption = model.generate_caption(
            image_tensor=image_tensor,
            vocabulary_tokenizer=tokenizer,
            max_length=50
        )
    return caption

In [25]:
def get_transformer_model_for_inference(vocab_size, checkpoint_path, model_type):
    print(f"Checkpoint yükleniyor: {checkpoint_path}")
    try:
        checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    except FileNotFoundError:
        print(f"HATA: Checkpoint dosyası bulunamadı: {checkpoint_path}")
        return None, None
    except Exception as e:
        print(f"HATA: Checkpoint yüklenirken bir sorun oluştu (torch.load): {e}")
        return None, None

    d_model = checkpoint.get('d_model', 256)
    nhead = checkpoint.get('nhead', 8)
    num_encoder_layers = checkpoint.get('num_encoder_layers', 3)
    num_decoder_layers = checkpoint.get('num_decoder_layers', 3)
    dim_feedforward = checkpoint.get('dim_feedforward', 1024)
    transformer_dropout = checkpoint.get('transformer_dropout', 0.1)
    actual_vocab_size = checkpoint.get('vocab_size', vocab_size) 


    print("Transformer modeli oluşturuluyor...")
    print(f"Kullanılan parametreler: d_model={d_model}, nhead={nhead}, enc_layers={num_encoder_layers}, dec_layers={num_decoder_layers}, ffw_dim={dim_feedforward}, vocab_size={actual_vocab_size}")

    model = CaptioningTransformer(
        num_tokens_vocab=actual_vocab_size,
        d_model=d_model,
        nhead=nhead,
        num_encoder_layers=num_encoder_layers,
        num_decoder_layers=num_decoder_layers,
        dim_feedforward=dim_feedforward,
        dropout=transformer_dropout,
        max_seq_len=100,
        cnn_type=model_type
    ).to(DEVICE)
    
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except Exception as e:
        print(f"HATA: Model state_dict yüklenirken sorun oluştu: {e}")
        print("Model mimarisi ile checkpoint'teki ağırlıklar uyumsuz olabilir.")
        return None, vocab 
        
    model.eval()
    print("Model başarıyla yüklendi ve evaluation moduna alındı.")
    return model

In [26]:
def get_model_for_inference(vocab_size, vocab, checkpoint_path, model_type, isTokenizer=False):
    print("Model oluşturuluyor...")
    if "REP" in model_type:
        model = CNNtoRNN_REP(EMBED_SIZE_LSTM, HIDDEN_SIZE_LSTM, vocab_size, NUM_LAYERS_LSTM, model_type).to(DEVICE)
    elif "inception" in model_type:
        checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
        embed_size  = checkpoint['embed_size']
        hidden_size = checkpoint['hidden_size']
        num_layers  = checkpoint['num_layers']
        vocab_size  = checkpoint['vocab_size']
        model = CNNtoRNN(
        embed_size  = embed_size,
        hidden_size = hidden_size,
        vocab_size  = vocab_size,
        vocab = vocab,
        num_layers  = num_layers,
        cnn_type    = model_type      
        ).to(DEVICE)
        sd = checkpoint['state_dict']
        model.load_state_dict(sd)
    else:
        if isTokenizer:
            model = CNNtoRNN_TOK(768, 768, len(vocabT), vocabT, NUM_LAYERS_LSTM, model_type).to(DEVICE)
        else:
            model = CNNtoRNN(EMBED_SIZE_LSTM, HIDDEN_SIZE_LSTM, vocab_size, vocab, NUM_LAYERS_LSTM, model_type).to(DEVICE)
    
    print(f"Checkpoint yükleniyor: {checkpoint_path}")
    try:
        if "inception" not in model_type:
           checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
           sd = checkpoint["state_dict"]  
           model.load_state_dict(sd)
            
    except FileNotFoundError:
        print(f"HATA: Checkpoint dosyası bulunamadı: {checkpoint_path}")
        return None
    except Exception as e:
        print(f"HATA: Checkpoint yüklenirken bir sorun oluştu: {e}")
        return None
        
    model.eval()
    print("Model başarıyla yüklendi ve evaluation moduna alındı.")
    return model

In [27]:
# GET MODELS
if not vocabulary or len(vocabulary) <= 4:
    print("Kelime dağarcığı oluşturulamadı veya çok küçük. Test işlemi yapılamıyor.")
else:
    VOCAB_SIZE = len(vocabulary)
    model_bnl_mini = get_model_for_inference(VOCAB_SIZE, vocabulary, BNL_MINI_CHECKPOINT, "inceptionv3")
    model_bnl_large = get_model_for_inference(VOCAB_SIZE, vocabulary, BNL_LARGE_CHECKPOINT, "resnet50")
    model_bnl_medium = get_model_for_inference(VOCAB_SIZE, vocabulary, BNL_MID_CHECKPOINT, "vgg16")
    model_bnl_large_rep = get_model_for_inference(len(rep_vocabulary), vocabulary, BNL_LARGE_REP_CHECKPOINT, "resnet50-REP")
    model_bnl_large_turbo = get_model_for_inference(len(vocabT), vocabT, BNL_LARGE_TURBO_CHECKPOINT, "tokenizer", isTokenizer=True)
    
    model_bnt_large = get_transformer_model_for_inference(VOCAB_SIZE, BNT_LARGE_CHECKPOINT, "resnet50")
    model_bnt_large_turbo = get_tokenizer_model_for_inference(BNT_LARGE_TURBO_CHECKPOINT, vocabT, "resnet50", DEVICE)
    model_bnt_large_ultra = get_tokenizer_model_for_inference(BNT_LARGE_ULTRA_CHECKPOINT, vocabT, "inceptionv3", DEVICE)
        

models_dict = {
    "BNL-Mini": (model_bnl_mini, "inceptionv3"),
    "BNL-Medium": (model_bnl_medium, "vgg168"),
    "BNL-Large": (model_bnl_large, "resnet50"),
    "BNL-Large-REP": (model_bnl_large_rep, "resnet50"),
    "BNL-Large-Turbo": (model_bnl_large_turbo, "resnet50"),
    "BNT-Large": (model_bnt_large, "resnet50"),
    "BNT-Large-Turbo": (model_bnt_large_turbo, "resnet50"),
    "BNT-Large-Ultra": (model_bnt_large_ultra, "inceptionv3")
}

Model oluşturuluyor...


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 175MB/s]  


Checkpoint yükleniyor: /kaggle/input/inception_lstm_vocab/pytorch/default/1/inception_lstm_vocab.pth
Model başarıyla yüklendi ve evaluation moduna alındı.
Model oluşturuluyor...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 178MB/s] 


Checkpoint yükleniyor: /kaggle/input/resnet50_lstm_combined/pytorch/default/1/resnet50_combined.pth
Model başarıyla yüklendi ve evaluation moduna alındı.
Model oluşturuluyor...


...

Checkpoint yükleniyor: /kaggle/input/vgg16_lstm_combined/pytorch/default/1/vgg16_combined_model.pth
Model başarıyla yüklendi ve evaluation moduna alındı.
Model oluşturuluyor...
Checkpoint yükleniyor: /kaggle/input/rep_penalty/pytorch/default/1/mscoco_checkpoint_best(7).pth
Model başarıyla yüklendi ve evaluation moduna alındı.
Model oluşturuluyor...


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 203MB/s] 


Checkpoint yükleniyor: /kaggle/input/resnet_lstm_tokenizer/pytorch/default/1/resnet_lstm_tokenizer.pth
Model başarıyla yüklendi ve evaluation moduna alındı.
Checkpoint yükleniyor: /kaggle/input/resnet50_transformer_vocab/pytorch/default/1/mscoco_transformer_checkpoint_best(2).pth
Transformer modeli oluşturuluyor...
Kullanılan parametreler: d_model=256, nhead=8, enc_layers=3, dec_layers=3, ffw_dim=1024, vocab_size=23233
Model başarıyla yüklendi ve evaluation moduna alındı.
[INFO] Checkpoint'ten config yüklendi.
[✓] Model ağırlıkları başarıyla yüklendi.
[INFO] Checkpoint'ten config yüklendi.


/usr/local/lib/python3.11/dist-packages/torchvision/models/feature_extraction.py:175: UserWarning: NOTE: The nodes obtained by tracing the model in eval mode are a subsequence of those obtained in train mode. When choosing nodes for feature extraction, you may need to specify output nodes for train and eval mode separately.
  warnings.warn(msg + suggestion_msg)


[✓] Model ağırlıkları başarıyla yüklendi.


## before-inference methods

In [28]:
def transform_image_for_inference(image_path, model_type):
    if model_type == "resnet50":
        weights = ResNet50_Weights.DEFAULT
    elif model_type == "vgg16":
         weights = VGG16_Weights.DEFAULT
    else:
         weights = Inception_V3_Weights.DEFAULT

    preprocessed = weights.transforms(antialias=True) # Bu, resize, crop, ToTensor, Normalize içerir
    
    try:
        image = Image.open(image_path).convert("RGB")
        transformed_image = preprocessed(image)
        return transformed_image.unsqueeze(0) # Batch boyutu ekle (1, C, H, W)
    except FileNotFoundError:
        print(f"HATA: Görüntü dosyası bulunamadı: {image_path}")
        return None
    except Exception as e:
        print(f"HATA: Görüntü işlenirken bir sorun oluştu: {e}")
        return None


In [29]:
def generate_caption_from_image_path(image_path_to_test, model, model_type, vocab):
    if model is None or vocab is None:
        print("Model veya kelime dağarcığı yüklenemedi. İşlem iptal edildi.")
        return

    print(f"\nTest edilecek görüntü: {image_path_to_test}")
    image_tensor = transform_image_for_inference(image_path_to_test, model_type)

    if image_tensor is None:
        return

    image_tensor = image_tensor.to(DEVICE)
    
    print("Altyazı üretiliyor...")
    
    predicted_caption_tokens = model.caption_image(image_tensor, vocab, max_length=50)

    try:
        eos_index = predicted_caption_tokens.index("<EOS>")
        predicted_caption_tokens = predicted_caption_tokens[:eos_index]
    except ValueError:
        pass 

    generated_caption = " ".join(predicted_caption_tokens)
    print(f"Üretilen Altyazı: {generated_caption}")
    return generated_caption

In [30]:
def generate_caption_for_image_path_transformers(image_path_to_test, model, model_type, vocab):
    if model is None or vocab is None:
        print("Model veya kelime dağarcığı yüklenemedi. İşlem iptal edildi.")
        return None

    print(f"\nTest edilecek görüntü: {image_path_to_test}")
    image_tensor = transform_image_for_inference(image_path_to_test, model_type)

    if image_tensor is None:
        return None

    image_tensor = image_tensor.to(DEVICE)
    
    print("Altyazı üretiliyor (Transformer ile)...")
    
    try:
        sos_token_id = vocab.stoi["<SOS>"]
        eos_token_id = vocab.stoi["<EOS>"]
        pad_token_id = vocab.stoi["<PAD>"]
    except AttributeError:
        print("HATA: Kelime dağarcığı nesnesinde 'stoi' özelliği bulunamadı. Lütfen Vocabulary sınıfınızı veya reconstruct_vocab_from_checkpoint_data fonksiyonunu kontrol edin.")
        return None
    except KeyError as e:
        print(f"HATA: Özel token ({e}) kelime dağarcığında bulunamadı.")
        return None

    predicted_caption_tokens = model.generate_caption(
        image_tensor, 
        vocab,
        start_token_id=sos_token_id,
        end_token_id=eos_token_id,
        pad_token_id=pad_token_id,
        max_length=50
    )

    try:
      eos_index = predicted_caption_tokens.index("<EOS>")
      predicted_caption_tokens = predicted_caption_tokens[:eos_index]
    except ValueError:
        pass 
    generated_caption = " ".join(predicted_caption_tokens)
    print(f"Üretilen Altyazı: {generated_caption}")
    return generated_caption

## gradio fn

In [31]:
def gradio_generate_caption(model_choice, image_path):
    selected_model, model_type = models_dict.get(model_choice, (None, None))
    if selected_model is None:
        return f"⚠️ Geçersiz model seçimi: {model_choice}"

    if selected_model is model_bnl_large_rep:
        caption = generate_caption_from_image_path(image_path, selected_model, model_type, rep_vocabulary)
    elif selected_model is model_bnt_large:
        caption = generate_caption_for_image_path_transformers(image_path, selected_model, model_type, vocabulary)
    elif selected_model is model_bnt_large_turbo or selected_model is model_bnt_large_ultra:
        caption = generate_caption_from_image_path_T(image_path, selected_model, vocabT, DEVICE)
    elif selected_model is model_bnl_large_turbo:
        caption = generate_caption_from_image_path(image_path, selected_model, model_type, vocabT)
    else:
        caption = generate_caption_from_image_path(image_path, selected_model, model_type, vocabulary)
        
    if not caption:
        return "Altyazı üretilemedi ya da boş döndü."
    return caption


## ngrok - server

In [33]:
ngrok.kill()
demo = gr.Interface(
    fn=gradio_generate_caption,
    inputs=[
        gr.Dropdown(choices=list(models_dict.keys()),   
                    value="BNL-Large",                   
                    label="Choose a model"),
        gr.Image(type="filepath", label="Upload a picture")
    ],
    outputs=gr.Textbox(label="Produced caption"),
    title="Automatic Title Extraction Using Deep Learning",
    description="""
    Upload a picture to get it captioned.
    """,
    allow_flagging="never"
)

public_url = ngrok.connect(addr="7860")
print("🔗 ngrok public URL:", public_url)

def run_gradio():
    demo.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=False
    )

thread = threading.Thread(target=run_gradio, daemon=True)
thread.start()


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


🔗 ngrok public URL: NgrokTunnel: "https://36940a0ba094.ngrok-free.app" -> "http://localhost:7860"
* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.



Test edilecek görüntü: /tmp/gradio/fc8f49a596d176e3873c4e55c3de534b54894f142f431b8428d70500213f47fb/29503-TZi1PRD7j3MIPEqJ.jpg
Altyazı üretiliyor...
Üretilen Altyazı: bir banyoda tuvalet ve lavabo

Test edilecek görüntü: /tmp/gradio/71c4edbdbba2b7dc080294d0778b308df6a6d8d5339bafaa2aad62b7dd504705/IMG_1240.JPG
Altyazı üretiliyor...
Üretilen Altyazı: bir adam bir masada oturuyor

Test edilecek görüntü: /tmp/gradio/71c4edbdbba2b7dc080294d0778b308df6a6d8d5339bafaa2aad62b7dd504705/IMG_1240.JPG
Altyazı üretiliyor...
Üretilen Altyazı: Bir adam bir video oyunu oynuyor .

Test edilecek görüntü: /tmp/gradio/71c4edbdbba2b7dc080294d0778b308df6a6d8d5339bafaa2aad62b7dd504705/IMG_1240.JPG
Altyazı üretiliyor (Transformer ile)...


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Üretilen Altyazı: bir adam bir makas tutuyor ve bir şeyler yiyor

Test edilecek görüntü: /tmp/gradio/71c4edbdbba2b7dc080294d0778b308df6a6d8d5339bafaa2aad62b7dd504705/IMG_1240.JPG
Altyazı üretiliyor (Transformer ile)...
Üretilen Altyazı: bir adam bir makas tutuyor ve bir şeyler yiyor

Test edilecek görüntü: /tmp/gradio/dd09b2ebcccdb6399a42b8f28daa1d8bf6d023fbe7b34e117e44682074181b04/IMG_3075.JPG
Altyazı üretiliyor...
Üretilen Altyazı: bir sahilde uçurtma uçuran bir kişi

Test edilecek görüntü: /tmp/gradio/2866f401f3c2adde6b403a5a24ebcc002d0645c1000767c539f330260d4c05bb/IMG_9822.JPG
Altyazı üretiliyor...
Üretilen Altyazı: bir tekne suda bir tekne ve bir tekne

Test edilecek görüntü: /tmp/gradio/2866f401f3c2adde6b403a5a24ebcc002d0645c1000767c539f330260d4c05bb/IMG_9822.JPG
Altyazı üretiliyor...
Üretilen Altyazı: bir su kütlesinde bir tekne
